In [1]:
# Using Google Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
# from lightgbm import LGBMRegressor
# from skorch import NeuralNetRegressor
from sklearn.preprocessing import PolynomialFeatures, OneHotEncoder, minmax_scale
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neural_network import MLPRegressor
# from sklearn.feature_selection import RFECV
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
# from sklearn.experimental import enable_iterative_imputer
# from sklearn.impute import IterativeImputer
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, cross_validate
# from sklearn.model_selection import KFold

df = pd.read_csv("/content/drive/MyDrive/4080_job_length_prediction/mlperf_dataset.csv") # Change directory if needed
df = df.drop(['epoch_count'], axis=1) # epoch_count seems to have no value captured or be wildly large, batch_size is more reliable
df.head(5)

,model_name,data_name,gpu_name,gpu_count,cpu_count,cpu_core_count,cpu_mem_gb,batch_size,time_min,gpu_mem_gb,type,gpu_speed_rank,model_complexity,data_size_rank
0,ResNet50,ImageNet,NVIDIA H100-SXM5-80GB,64,16,448,16000,3200,4.246577,684.166480,train,7,2,8
1,R-GAT,IGBH-Full,NVIDIA H200-SXM5-141GB,8,2,56,2000,16384,9.415868,150.754820,train,3,5,12
2,3D U-Net,KiTS19,NVIDIA H100-SXM5-80GB,72,18,504,18000,56,2.553292,769.687290,train,7,4,4
3,ResNet50,ImageNet,NVIDIA L40S,8,4,64,2000,1600,51.380033,96.611598,train,10,2,8
4,Llama 2 70B,SCROLLS GovReport,NVIDIA L40S,64,32,512,16000,8,22.584457,772.892787,train,10,12,2


In [ ]:
### TRAINING JOBS ###
df_train = df[df['type'] == 'train'].drop(['type'], axis=1)

X_cat = df_train[['model_name','data_name','gpu_name']]
X_num = df_train[['gpu_count','cpu_count','cpu_mem_gb','cpu_core_count','batch_size','gpu_mem_gb','gpu_speed_rank','model_complexity', 'data_size_rank']]

onehot_encoder = OneHotEncoder(handle_unknown='ignore')
X_cat_encoded_array = onehot_encoder.fit_transform(X_cat)
X_cat_encoded = pd.DataFrame(
  X_cat_encoded_array.toarray(),
  index=X_cat.index,
  columns=onehot_encoder.get_feature_names_out(X_cat.columns)
)

X_num_scaled = pd.DataFrame(
  minmax_scale(X_num),
  index=X_num.index,
  columns=X_num.columns
)

X_all = pd.concat([X_num_scaled, X_cat_encoded], axis=1)
y_all = np.log1p(df_train["time_min"]).values # Use log-transform to reduce variance

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2, random_state=42) # Only used for GridSearch (test data unused)

## GRIDSEARCHING ##
# --- Random Forest Ensemble Regression Trees --- #
rf_model = RandomForestRegressor(random_state=42)
grid_rf = GridSearchCV(rf_model, {"max_depth": [3, 5, 7, 10], "min_samples_split": [2, 3, 4]}, cv=5, scoring='neg_mean_squared_error')
grid_rf.fit(X_train, y_train)
best_rf_model = grid_rf.best_estimator_
print("---TRAINING JOBS---\n* GRID SEARCH RESULTS (score is RMSE) *")
print(f"RANDOM FOREST:\nBest params: {grid_rf.best_params_}\nBest score: {np.sqrt(-grid_rf.best_score_)}\n")

# --- XGBoost Gradient-Boosted Regression Trees --- #
xgb_model = GradientBoostingRegressor(random_state=42)
grid_xgb = GridSearchCV(xgb_model,{"max_depth": [3, 5, 7, 10], "learning_rate": [0.01, 0.05, 0.1], "min_samples_split": [2, 3, 4]}, cv=5, scoring='neg_mean_squared_error')
grid_xgb.fit(X_train, y_train)
best_xgb_model = grid_xgb.best_estimator_
print(f"XGBOOST:\nBest params: {grid_xgb.best_params_}\nBest score: {np.sqrt(-grid_xgb.best_score_)}\n")

# --- MLP Neural Network --- #
mlp_model = MLPRegressor(learning_rate="adaptive", activation="relu", max_iter=3000, random_state=42)
grid_mlp = GridSearchCV(mlp_model, {
  "hidden_layer_sizes": [(128,64), (32,128,64), (64,128,32), (32,64,128,64), (32,64,128,32), (32,64,128,256,64)],
  "alpha": [0.0001, 0.001, 0.01, 0.1],
  "learning_rate_init": [0.001, 0.01, 0.1]
}, cv=5, scoring='neg_mean_squared_error')
grid_mlp.fit(X_train, y_train)
best_mlp_model = grid_mlp.best_estimator_
print(f"MLP NN:\nBest params: {grid_mlp.best_params_}\nBest score: {np.sqrt(-grid_mlp.best_score_)}")

---TRAINING JOBS---
* GRID SEARCH RESULTS (score is RMSE) *
RANDOM FOREST:
Best params: {'max_depth': 10, 'min_samples_split': 2}
Best score: 0.42572034357341254

XGBOOST:
Best params: {'learning_rate': 0.1, 'max_depth': 3, 'min_samples_split': 4}
Best score: 0.3825214795081191

MLP NN:
Best params: {'alpha': 0.1, 'hidden_layer_sizes': (32, 64, 128, 64), 'learning_rate_init': 0.001}
Best score: 0.5172223327685161


In [5]:
## TRAIN KFOLD CROSSVAL SCORING ##
print("* CROSS VAL RESULTS *")

# --- Random Forest --- #
rf_r2_scores = cross_val_score(best_rf_model, X_all, y_all, cv=10, scoring='r2')
print(f"Random Forest:\nScore per fold (R^2): {rf_r2_scores}\nMean score (R^2): {np.mean(rf_r2_scores):.2f}")
rf_rmse_scores = cross_val_score(best_rf_model, X_all, y_all, cv=10, scoring='neg_root_mean_squared_error')
print(f"Score per fold (RMSE): {rf_rmse_scores}\nMean score (RMSE): {np.mean(rf_rmse_scores*(-1)):.2f}")
rf_mae_scores = cross_val_score(best_rf_model, X_all, y_all, cv=10, scoring='neg_mean_absolute_error')
print(f"Score per fold (MAE): {rf_mae_scores}\nMean score (MAE): {np.mean(rf_mae_scores*(-1)):.2f}\n")

# --- XGBoost --- #
xgb_r2_scores = cross_val_score(best_xgb_model, X_all, y_all, cv=10, scoring='r2')
print(f"XGBoost:\nScore per fold (R^2): {xgb_r2_scores}\nMean score (R^2): {np.mean(xgb_r2_scores):.2f}")
xgb_rmse_scores = cross_val_score(best_xgb_model, X_all, y_all, cv=10, scoring='neg_root_mean_squared_error')
print(f"Score per fold (RMSE): {xgb_rmse_scores}\nMean score (RMSE): {np.mean(xgb_rmse_scores*(-1)):.2f}")
xgb_mae_scores = cross_val_score(best_xgb_model, X_all, y_all, cv=10, scoring='neg_mean_absolute_error')
print(f"Score per fold (MAE): {xgb_mae_scores}\nMean score (MAE): {np.mean(xgb_mae_scores*(-1)):.2f}\n")

# --- MLP Neural Network --- #
mlp_r2_scores = cross_val_score(best_mlp_model, X_all, y_all, cv=10, scoring='r2')
print(f"MLP NN:\nScore per fold (R^2): {mlp_r2_scores}\nMean score (R^2): {np.mean(mlp_r2_scores):.2f}")
mlp_rmse_scores = cross_val_score(best_mlp_model, X_all, y_all, cv=10, scoring='neg_root_mean_squared_error')
print(f"Score per fold (RMSE): {mlp_rmse_scores}\nMean score (RMSE): {np.mean(mlp_rmse_scores*(-1)):.2f}")
mlp_mae_scores = cross_val_score(best_mlp_model, X_all, y_all, cv=10, scoring='neg_mean_absolute_error')
print(f"Score per fold (MAE): {mlp_mae_scores}\nMean score (MAE): {np.mean(mlp_mae_scores*(-1)):.2f}")

* CROSS VAL RESULTS *
Random Forest:
Score per fold (R^2): [0.51652502 0.80398945 0.91665493 0.96976056 0.97258963 0.91702952
 0.96711111 0.88155581 0.92314211 0.77727913]
Mean score (R^2): 0.86
Score per fold (RMSE): [-0.77517265 -0.2574187  -0.26868499 -0.19318236 -0.14992966 -0.37251924
 -0.19727319 -0.40745409 -0.40234546 -0.57051866]
Mean score (RMSE): 0.36
Score per fold (MAE): [-0.6490249  -0.20648068 -0.19886354 -0.16646965 -0.12245278 -0.29046778
 -0.14915834 -0.24622508 -0.33810094 -0.37531229]
Mean score (MAE): 0.27

XGBoost:
Score per fold (R^2): [0.72195691 0.94367341 0.90316054 0.96402255 0.97000986 0.92764482
 0.98196769 0.87026034 0.96209037 0.91278954]
Mean score (R^2): 0.92
Score per fold (RMSE): [-0.58785132 -0.13799301 -0.2896207  -0.21071516 -0.15682646 -0.34787379
 -0.14607271 -0.42644024 -0.28257235 -0.35700449]
Mean score (RMSE): 0.29
Score per fold (MAE): [-0.45274233 -0.11012344 -0.2040351  -0.1639607  -0.12200825 -0.23781446
 -0.0931886  -0.23645087 -0.233238

In [ ]:
### INFERENCE JOBS ###
df_inf = df[df['type'] == 'inference'].drop(['type', 'batch_size'], axis=1)

X_cat = df_inf[['model_name','data_name','gpu_name']]
X_num = df_inf[['gpu_count','cpu_count','cpu_mem_gb','gpu_mem_gb','gpu_speed_rank','model_complexity','cpu_core_count','data_size_rank']]

onehot_encoder = OneHotEncoder(handle_unknown='ignore')
X_cat_encoded_array = onehot_encoder.fit_transform(X_cat)
X_cat_encoded = pd.DataFrame(
  X_cat_encoded_array.toarray(),
  index=X_cat.index,
  columns=onehot_encoder.get_feature_names_out(X_cat.columns)
)

X_num_scaled = pd.DataFrame(
  minmax_scale(X_num),
  index=X_num.index,
  columns=X_num.columns
)

X_all = pd.concat([X_num_scaled, X_cat_encoded], axis=1)
y_all = np.log1p(df_inf["time_min"]).values # Use log-transform to reduce variance

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2, random_state=42) # Only used for GridSearch (test data unused)

## GRIDSEARCHING ##
# --- Random Forest Ensemble Regression Trees --- #
rf_model = RandomForestRegressor(random_state=42)
grid_rf = GridSearchCV(rf_model, {"max_depth": [3, 5, 7, 10], "min_samples_split": [2, 3, 4]}, cv=5, scoring='neg_mean_squared_error')
grid_rf.fit(X_train, y_train)
best_rf_model = grid_rf.best_estimator_
print("---INFERENCE JOBS---\n* GRID SEARCH RESULTS (score is RMSE) *")
print(f"RANDOM FOREST:\nBest params: {grid_rf.best_params_}\nBest score: {np.sqrt(-grid_rf.best_score_)}\n")

# --- XGBoost Gradient-Boosted Regression Trees --- #
xgb_model = GradientBoostingRegressor(random_state=42)
grid_xgb = GridSearchCV(xgb_model,{"max_depth": [3, 5, 7, 10], "learning_rate": [0.01, 0.05, 0.1], "min_samples_split": [2, 3, 4]}, cv=5, scoring='neg_mean_squared_error')
grid_xgb.fit(X_train, y_train)
best_xgb_model = grid_xgb.best_estimator_
print(f"XGBOOST:\nBest params: {grid_xgb.best_params_}\nBest score: {np.sqrt(-grid_xgb.best_score_)}\n")

# --- MLP Neural Network --- #
mlp_model = MLPRegressor(learning_rate="adaptive", activation="relu", max_iter=3000, random_state=42)
grid_mlp = GridSearchCV(mlp_model, {
  "hidden_layer_sizes": [(128,64), (32,128,64), (64,128,32), (32,64,128,64), (32,64,128,32), (32,64,128,256,64)],
  "alpha": [0.0001, 0.001, 0.01, 0.1],
  "learning_rate_init": [0.001, 0.01, 0.1]
}, cv=5, scoring='neg_mean_squared_error')
grid_mlp.fit(X_train, y_train)
best_mlp_model = grid_mlp.best_estimator_
print(f"MLP NN:\nBest params: {grid_mlp.best_params_}\nBest score: {np.sqrt(-grid_mlp.best_score_)}")

---INFERENCE JOBS---
* GRID SEARCH RESULTS (score is RMSE) *
RANDOM FOREST:
Best params: {'max_depth': 5, 'min_samples_split': 2}
Best score: 0.978762852249412

XGBOOST:
Best params: {'learning_rate': 0.05, 'max_depth': 7, 'min_samples_split': 4}
Best score: 0.8593941387587601

MLP NN:
Best params: {'alpha': 0.0001, 'hidden_layer_sizes': (32, 64, 128, 256, 64), 'learning_rate_init': 0.001}
Best score: 0.9273106002723032


In [8]:
## INFERENCE KFOLD CROSSVAL SCORING ##
print("* CROSS VAL RESULTS *")

# --- Random Forest --- #
rf_r2_scores = cross_val_score(best_rf_model, X_all, y_all, cv=10, scoring='r2')
print(f"Random Forest:\nScore per fold (R^2): {rf_r2_scores}\nMean score (R^2): {np.mean(rf_r2_scores):.2f}")
rf_rmse_scores = cross_val_score(best_rf_model, X_all, y_all, cv=10, scoring='neg_root_mean_squared_error')
print(f"Score per fold (RMSE): {rf_rmse_scores}\nMean score (RMSE): {np.mean(rf_rmse_scores*(-1)):.2f}")
rf_mae_scores = cross_val_score(best_rf_model, X_all, y_all, cv=10, scoring='neg_mean_absolute_error')
print(f"Score per fold (MAE): {rf_mae_scores}\nMean score (MAE): {np.mean(rf_mae_scores*(-1)):.2f}\n")

# --- XGBoost --- #
xgb_r2_scores = cross_val_score(best_xgb_model, X_all, y_all, cv=10, scoring='r2')
print(f"XGBoost:\nScore per fold (R^2): {xgb_r2_scores}\nMean score (R^2): {np.mean(xgb_r2_scores):.2f}")
xgb_rmse_scores = cross_val_score(best_xgb_model, X_all, y_all, cv=10, scoring='neg_root_mean_squared_error')
print(f"Score per fold (RMSE): {xgb_rmse_scores}\nMean score (RMSE): {np.mean(xgb_rmse_scores*(-1)):.2f}")
xgb_mae_scores = cross_val_score(best_xgb_model, X_all, y_all, cv=10, scoring='neg_mean_absolute_error')
print(f"Score per fold (MAE): {xgb_mae_scores}\nMean score (MAE): {np.mean(xgb_mae_scores*(-1)):.2f}\n")

# --- MLP Neural Network --- #
mlp_r2_scores = cross_val_score(best_mlp_model, X_all, y_all, cv=10, scoring='r2')
print(f"MLP NN:\nScore per fold (R^2): {mlp_r2_scores}\nMean score (R^2): {np.mean(mlp_r2_scores):.2f}")
mlp_rmse_scores = cross_val_score(best_mlp_model, X_all, y_all, cv=10, scoring='neg_root_mean_squared_error')
print(f"Score per fold (RMSE): {mlp_rmse_scores}\nMean score (RMSE): {np.mean(mlp_rmse_scores*(-1)):.2f}")
mlp_mae_scores = cross_val_score(best_mlp_model, X_all, y_all, cv=10, scoring='neg_mean_absolute_error')
print(f"Score per fold (MAE): {mlp_mae_scores}\nMean score (MAE): {np.mean(mlp_mae_scores*(-1)):.2f}")

* CROSS VAL RESULTS *
Random Forest:
Score per fold (R^2): [ 0.7701214   0.65328338  0.90725571 -0.20440823  0.6757042   0.86823185
  0.7535201   0.86368769  0.89558999  0.8296047 ]
Mean score (R^2): 0.70
Score per fold (RMSE): [-0.57807215 -0.6098244  -0.31285752 -0.94862859 -0.77032732 -0.94385268
 -0.50969608 -0.47074087 -0.37854886 -0.70937471]
Mean score (RMSE): 0.62
Score per fold (MAE): [-0.43671352 -0.44276979 -0.21558742 -0.80439163 -0.63735457 -0.64679719
 -0.36937628 -0.34284032 -0.28548292 -0.60684866]
Mean score (MAE): 0.48

XGBoost:
Score per fold (R^2): [ 0.68584625  0.71412135  0.8808642  -0.37324627  0.56086876  0.88356406
  0.68164704  0.76221611  0.88894132  0.6999094 ]
Mean score (R^2): 0.64
Score per fold (RMSE): [-0.67577777 -0.5537431  -0.35458811 -1.01293963 -0.89639991 -0.88724285
 -0.57926193 -0.62173537 -0.39041558 -0.94139761]
Mean score (RMSE): 0.69
Score per fold (MAE): [-0.43890209 -0.37897693 -0.24664746 -0.77971109 -0.70544121 -0.57021397
 -0.36551527 -

In [ ]:
### COMBINED MODEL ###
X_cat = df[['model_name','data_name','gpu_name','type']]
X_num = df[['gpu_count','cpu_count','cpu_mem_gb','cpu_core_count','batch_size','gpu_mem_gb','gpu_speed_rank','model_complexity', 'data_size_rank']]

onehot_encoder = OneHotEncoder(handle_unknown='ignore')
X_cat_encoded_array = onehot_encoder.fit_transform(X_cat)
X_cat_encoded = pd.DataFrame(
  X_cat_encoded_array.toarray(),
  index=X_cat.index,
  columns=onehot_encoder.get_feature_names_out(X_cat.columns)
)

X_num_scaled = pd.DataFrame(
  minmax_scale(X_num),
  index=X_num.index,
  columns=X_num.columns
)

X_all = pd.concat([X_num_scaled, X_cat_encoded], axis=1)
y_all = np.log1p(df["time_min"]).values # Use log-transform to reduce variance

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2, random_state=42) # Only used for GridSearch (test data unused)

## GRIDSEARCHING ##
# --- Random Forest Ensemble Regression Trees --- #
rf_model = RandomForestRegressor(random_state=42)
grid_rf = GridSearchCV(rf_model, {"max_depth": [3, 5, 7, 10], "min_samples_split": [2, 3, 4]}, cv=5, scoring='neg_mean_squared_error')
grid_rf.fit(X_train, y_train)
best_rf_model = grid_rf.best_estimator_
print("---COMBINED JOBS---\n* GRID SEARCH RESULTS (score is RMSE) *")
print(f"RANDOM FOREST:\nBest params: {grid_rf.best_params_}\nBest score: {np.sqrt(-grid_rf.best_score_)}\n")

# --- XGBoost Gradient-Boosted Regression Trees --- #
xgb_model = GradientBoostingRegressor(random_state=42)
grid_xgb = GridSearchCV(xgb_model,{"max_depth": [3, 5, 7, 10], "learning_rate": [0.01, 0.05, 0.1], "min_samples_split": [2, 3, 4]}, cv=5, scoring='neg_mean_squared_error')
grid_xgb.fit(X_train, y_train)
best_xgb_model = grid_xgb.best_estimator_
print(f"XGBOOST:\nBest params: {grid_xgb.best_params_}\nBest score: {np.sqrt(-grid_xgb.best_score_)}\n")

# --- MLP Neural Network --- #
mlp_model = MLPRegressor(learning_rate="adaptive", activation="relu", max_iter=3000, random_state=42)
grid_mlp = GridSearchCV(mlp_model, {
  "hidden_layer_sizes": [(128,64), (32,128,64), (64,128,32), (32,64,128,64), (32,64,128,32), (32,64,128,256,64)],
  "alpha": [0.0001, 0.001, 0.01, 0.1],
  "learning_rate_init": [0.001, 0.01, 0.1]
}, cv=5, scoring='neg_mean_squared_error')
grid_mlp.fit(X_train, y_train)
best_mlp_model = grid_mlp.best_estimator_
print(f"MLP NN:\nBest params: {grid_mlp.best_params_}\nBest score: {np.sqrt(-grid_mlp.best_score_)}")

---COMBINED JOBS---
* GRID SEARCH RESULTS (score is RMSE) *
RANDOM FOREST:
Best params: {'max_depth': 10, 'min_samples_split': 2}
Best score: 0.6060340733137133

XGBOOST:
Best params: {'learning_rate': 0.1, 'max_depth': 3, 'min_samples_split': 4}
Best score: 0.529661101270856

MLP NN:
Best params: {'alpha': 0.1, 'hidden_layer_sizes': (64, 128, 32), 'learning_rate_init': 0.01}
Best score: 0.6046293429602484


In [10]:
## COMBINED KFOLD CROSSVAL SCORING ##
print("* CROSS VAL RESULTS *")

# --- Random Forest --- #
rf_r2_scores = cross_val_score(best_rf_model, X_all, y_all, cv=10, scoring='r2')
print(f"Random Forest:\nScore per fold (R^2): {rf_r2_scores}\nMean score (R^2): {np.mean(rf_r2_scores):.2f}")
rf_rmse_scores = cross_val_score(best_rf_model, X_all, y_all, cv=10, scoring='neg_root_mean_squared_error')
print(f"Score per fold (RMSE): {rf_rmse_scores}\nMean score (RMSE): {np.mean(rf_rmse_scores*(-1)):.2f}")
rf_mae_scores = cross_val_score(best_rf_model, X_all, y_all, cv=10, scoring='neg_mean_absolute_error')
print(f"Score per fold (MAE): {rf_mae_scores}\nMean score (MAE): {np.mean(rf_mae_scores*(-1)):.2f}\n")

# --- XGBoost --- #
xgb_r2_scores = cross_val_score(best_xgb_model, X_all, y_all, cv=10, scoring='r2')
print(f"XGBoost:\nScore per fold (R^2): {xgb_r2_scores}\nMean score (R^2): {np.mean(xgb_r2_scores):.2f}")
xgb_rmse_scores = cross_val_score(best_xgb_model, X_all, y_all, cv=10, scoring='neg_root_mean_squared_error')
print(f"Score per fold (RMSE): {xgb_rmse_scores}\nMean score (RMSE): {np.mean(xgb_rmse_scores*(-1)):.2f}")
xgb_mae_scores = cross_val_score(best_xgb_model, X_all, y_all, cv=10, scoring='neg_mean_absolute_error')
print(f"Score per fold (MAE): {xgb_mae_scores}\nMean score (MAE): {np.mean(xgb_mae_scores*(-1)):.2f}\n")

# --- MLP Neural Network --- #
mlp_r2_scores = cross_val_score(best_mlp_model, X_all, y_all, cv=10, scoring='r2')
print(f"MLP NN:\nScore per fold (R^2): {mlp_r2_scores}\nMean score (R^2): {np.mean(mlp_r2_scores):.2f}")
mlp_rmse_scores = cross_val_score(best_mlp_model, X_all, y_all, cv=10, scoring='neg_root_mean_squared_error')
print(f"Score per fold (RMSE): {mlp_rmse_scores}\nMean score (RMSE): {np.mean(mlp_rmse_scores*(-1)):.2f}")
mlp_mae_scores = cross_val_score(best_mlp_model, X_all, y_all, cv=10, scoring='neg_mean_absolute_error')
print(f"Score per fold (MAE): {mlp_mae_scores}\nMean score (MAE): {np.mean(mlp_mae_scores*(-1)):.2f}")

* CROSS VAL RESULTS *
Random Forest:
Score per fold (R^2): [0.39401651 0.92303922 0.91566758 0.87274201 0.82512883 0.64340523
 0.63012859 0.79357434 0.69902019 0.90464782]
Mean score (R^2): 0.76
Score per fold (RMSE): [-0.73302524 -0.27885096 -0.34817417 -0.44500487 -0.56394705 -0.71070033
 -0.64579419 -0.96018857 -0.52159797 -0.51590601]
Mean score (RMSE): 0.57
Score per fold (MAE): [-0.50958307 -0.22903893 -0.25597835 -0.31105182 -0.43545628 -0.46875867
 -0.46516022 -0.67705349 -0.37546454 -0.41352786]
Mean score (MAE): 0.41

XGBoost:
Score per fold (R^2): [0.64785596 0.8703049  0.93443329 0.935958   0.90582831 0.77208095
 0.65530485 0.7674529  0.74937291 0.88516162]
Mean score (R^2): 0.81
Score per fold (RMSE): [-0.5587901  -0.36199219 -0.30700181 -0.31568552 -0.41384661 -0.56818446
 -0.62342805 -1.01913134 -0.47597179 -0.56617253]
Mean score (RMSE): 0.52
Score per fold (MAE): [-0.38161621 -0.26052518 -0.21222542 -0.22826743 -0.30959338 -0.41013847
 -0.45957905 -0.76073681 -0.379554